In [1]:
import re
import os
import numpy as np
import pandas as pd

#### Read Dataset
---

In [2]:
dataset = 'mr'
files = os.listdir('data/{}'.format(dataset))
files.remove('.ipynb_checkpoints')

In [3]:
# read mr dataset

def clean_str(s):
    s = s.decode('latin-1')
    s = re.sub(r'\r\n', '', s)
    return s
    
def read_dataset(dataset, file):
    with open('data/{}/{}'.format(dataset, file), 'rb') as f:
        result = f.readlines()
        
    result = map(clean_str, result)
    if 'label' in file:
        result = list(map(int, result))
    else:
        result = list(map(lambda x: x.split(), result))
    return result
 

In [4]:
text_train = read_dataset('mr', 'text_train.txt')
label_train = read_dataset('mr', 'label_train.txt')

text_test = read_dataset('mr', 'text_test.txt')
label_test = read_dataset('mr', 'label_test.txt')

In [5]:
text_train_str = list(map(lambda x: " ".join(x), text_train))
text_test_str = list(map(lambda x: " ".join(x), text_test))

train_df = pd.DataFrame({'text': text_train_str, 'label': label_train})
test_df = pd.DataFrame({'text': text_test_str, 'label': label_test})

train_df.to_csv('data/mr/train_df.tsv', index_label=False, sep='\t')
test_df.to_csv('data/mr/test_df.tsv', index_label=False, sep='\t')

In [6]:
train_df['train_mask'] = True
train_df['test_mask'] = False
test_df['test_mask'] = True
test_df['train_mask'] = False

all_df = pd.concat([train_df, test_df], axis=0)
all_df.reset_index(drop=True, inplace=True)
all_df.reset_index(inplace=True)
all_df.columns = ["doc_id"] + list(all_df.columns)[1:]
all_df.doc_id = all_df.doc_id.astype(str)
all_df.doc_id = 'doc_id_' + all_df.doc_id
all_df.to_csv('data/mr/raw/all_df_mask.tsv', index_label=False, sep='\t')

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [7]:
import itertools
import pandas as pd
import numpy as np

class Vocab(object):
  
  def __init__(self, L, doc_ids=None):    
    if isinstance(L[0], list):
      tokens = list(itertools.chain(*L))
      self.token_counts = pd.Series(tokens).value_counts().to_frame().sort_index(ascending=True)
      self.vocab = ["<unk>"] + self.token_counts.index.to_list()
    else:
      tokens = self.token_counts = pd.Series(L).value_counts().to_frame().sort_index(ascending=True)
      self.vocab = self.token_counts.index.to_list()
    self.vocab = list(set(self.vocab))
    self.vocab = sorted(self.vocab)
    if doc_ids is not None:
      self.vocab = doc_ids + self.vocab 
    
    self.w2i = dict(zip(self.vocab, range(len(self.vocab))))
    self.i2w = dict(zip(range(len(self.vocab)), self.vocab))

  def map_words2index(self, L):
    return list(map(lambda x: self.w2i[x] if x in self.w2i else self.w2i['unk'], L))

  def map_index2words(self, L):
    return list(map(lambda x: self.i2w[x], L))

  def map_dataset_words2index(self, L):
    return np.array(list(map(self.map_words2index, L)))

  def map_dataset_index2words(self, L):
    return np.array(list(map(self.map_index2words, L)))

  def get_counts(self):
    return self.token_counts

In [8]:
%time vocab = Vocab(text_train)
%time train_x = vocab.map_dataset_words2index(text_train)

CPU times: user 82.8 ms, sys: 0 ns, total: 82.8 ms
Wall time: 82.5 ms
CPU times: user 67.4 ms, sys: 0 ns, total: 67.4 ms
Wall time: 67.2 ms


In [9]:
counts = vocab.get_counts()
counts.sort_values(0, inplace=True)
counts.head()

,0
kirsten,1
mid-range,1
mid-section,1
mid-seventies,1
mid-to-low,1


#### Create Dataset

In [10]:
files

['train_df.tsv',
 'label_train.txt',
 'raw',
 'all_df_masks',
 'label_test.txt',
 'text_test.txt',
 'text_all.txt',
 'processed',
 'test_df.tsv',
 'text_train.txt']

In [58]:
import torch
import torch.nn as nn
from torch_geometric.data import InMemoryDataset
from torch_geometric.utils import to_undirected, is_undirected

EMBED_DIM = 100

class MRDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(MRDataset, self).__init__(root, transform, pre_transform)
#         self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def raw_file_names(self):
        return ['all_df_mask.tsv']
    
    @property
    def processed_file_names(self):
        return ['mr_train.pt']
    
    def process(self):
        df = pd.read_csv(self.raw_paths[0], sep="\t")
        
        texts = list(map(lambda x: x.split(), df.text.values))
        doc_ids = df.doc_id.values
        labels = df.label.values
        doc_label_dict = dict(zip(doc_ids, labels))
        is_train_dict = dict(zip(doc_ids, df.train_mask.values))
        vocab = Vocab(texts, doc_ids=df.doc_id.values.tolist())
        text_int = vocab.map_dataset_words2index(texts)
        
        # nodes_idx mapping will be the same as vocab.i2w
        self.embed = nn.Embedding(len(vocab.w2i), EMBED_DIM)
        nodes = self.embed(torch.tensor(range(len(vocab.w2i))))  # (vocab_size, EMBED_DIM)
        edge_index = []
        for i, row in df.iterrows():
            doc_id = vocab.w2i[row["doc_id"]]
            edges_ = zip([doc_id for _ in range(len(text_int[i]))], text_int[i])
            edge_index.extend(list(edges_))

        edge_index = list(set(edge_index))  # a word may occur mult times in a doc
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        edge_index = to_undirected(edge_index)

        # masks:
        train_mask = []
        test_mask = []
        for i in range(len(vocab.i2w)):
            w = vocab.i2w[i]
            if w in doc_label_dict:
                is_train = is_train_dict[w]
                train_mask.append(is_train)
                test_mask.append(not is_train)
            else:
                train_mask.append(False)
                test_mask.append(False)
        
        import pdb; pdb.set_trace()
        
        # --- build the data object here ---
        # build nodes
#         import pdb; pdb.set_trace()   
        
        # build edges
#         print(self.processed_paths)
#         torch.save((nodes, None), self.processed_paths[0])
        return texts


In [59]:
torch.tensor([(1,2), (3,2)])

tensor([[1, 2],
        [3, 2]])

In [60]:
all_df.head()

,doc_id,label,test_mask,text,train_mask
0,doc_id_0,1,False,'moore is like a progressive bull in a china s...,True
1,doc_id_1,1,False,idiotic and ugly .,True
2,doc_id_2,1,False,even if the naipaul original remains the real ...,True
3,doc_id_3,1,False,"the movie is amateurish , but it's a minor tre...",True
4,doc_id_4,1,False,some people march to the beat of a different d...,True


In [61]:
import collections

In [62]:
%time texts = MRDataset('data/mr')

Processing...
> <ipython-input-58-6ceb7312ee77>(67)process()
-> return texts


(Pdb)  sum(train_mask)


7108


(Pdb)  sum(test_mask)


3554


...

0


(Pdb)  sum(test_mask[7108:11111])


3554


(Pdb)  q


BdbQuit: 

In [15]:
texts

NameError: name 'texts' is not defined

In [ ]:
from torch_geometric.utils import to_undirected, is_undirected